In [53]:
from os import walk
import pandas as pd
import numpy as np

cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

f = []
for (dirpath, dirnames, filenames) in walk('../blend/'):
    f.extend(filenames)
    break

ff = []
for (dirpath, dirnames, filenames) in walk('../skolbachev/'):
    for d in dirnames:
        for (dirpath, dirnames2, filenames) in walk('../skolbachev/'+d):
            for qf in filenames:
                ff.append('../skolbachev/'+d+'/'+qf)
                
chen_sol = pd.read_csv('../cheng/ensemble/9/gru.info.dsfu.lower_model.ckpt-30.00-67290.valid').sort_values('id').reset_index(drop=True)                
chen_sol_ids = chen_sol['id'].values

fchen = []
for (dirpath, dirnames, filenames) in walk('../cheng/ensemble/9'):
    fchen.extend([q for q in filenames if q.endswith('.valid')])
    break 

train_idx = pd.read_csv('../input/train.csv')['id'].values
train = pd.read_csv('../input/train.csv').sort_values('id').reset_index(drop=True)

In [54]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import minmax_scale

train = train.loc[train['id'].isin(chen_sol_ids),:].sort_values('id').reset_index(drop=True)

oofs = []
onms = []

train_files = [q for q in f if q.startswith('train')]
for q in train_files:
    nm = q[6:-4]
    nf = pd.read_csv('../blend/'+q)
    nf = nf.loc[nf.id.isin(chen_sol_ids),:].sort_values('id').reset_index(drop=True)
    for c in cols:
        if 'identity_hate' in nf.columns:
            nf[c] = minmax_scale(nf[c])
        else:
            nf[c] = minmax_scale(nf[c+'_oof'])
            nf.drop([c+'_oof'],axis=1,inplace=True)
        #print(nm,c,roc_auc_score(train[c],nf[c]))
    if (nf.columns.tolist().index('id')==0):
        nf.columns = ['id'] + [nm+'_' + q for q in cols]
    else:
        nf.columns = [nm+'_' + q for q in cols] + ['id']
    print(nm, roc_auc_score(train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']],nf[[nm+'_toxic',nm+'_severe_toxic',nm+'_obscene',nm+'_threat',nm+'_insult',nm+'_identity_hate']]))
    onms.append(nm)
    oofs.append(nf)
    
sk_train = [q for q in ff if not q.endswith('test_X_pred.npy')]
suf = 'sk'
i = 0
for q in sk_train:
    nf = pd.DataFrame(np.load(q))
    nm = suf+str(i)
    nf.columns = [nm+'_'+q for q in cols]
    nf['id'] = train_idx
    nf = nf.loc[nf.id.isin(chen_sol_ids),:].sort_values('id').reset_index(drop=True)
    for c in cols:
        nf[nm+'_'+c] = minmax_scale(nf[nm+'_'+c])
    print(nm, roc_auc_score(train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']],nf[[nm+'_toxic',nm+'_severe_toxic',nm+'_obscene',nm+'_threat',nm+'_insult',nm+'_identity_hate']]))
    onms.append(nm)
    oofs.append(nf)
    i = i + 1
    
suf = 'chen'
i = 0
for q in fchen:
    nf = pd.read_csv('../cheng/ensemble/9/'+q)
    nm = suf+str(i)
    nf.columns = ['id'] + [nm+'_'+q for q in cols]
    nf = nf.sort_values('id').reset_index(drop=True)
    for c in cols:
        nf[nm+'_'+c] = minmax_scale(nf[nm+'_'+c])
    try:
        print(nm, roc_auc_score(
                    train[cols],
                    nf[[nm+'_toxic',nm+'_severe_toxic',nm+'_obscene',nm+'_threat',nm+'_insult',nm+'_identity_hate']]))
        onms.append(nm)
        oofs.append(nf)
    except:
        nf = nf.loc[nf['id'] != '0',:].reset_index(drop=True)
        print(nm, roc_auc_score(
                    train[cols],
                    nf[[nm+'_toxic',nm+'_severe_toxic',nm+'_obscene',nm+'_threat',nm+'_insult',nm+'_identity_hate']]))
        onms.append(nm)
        oofs.append(nf)
        pass
    i = i + 1    

_att 0.9886094397266466
2ycnn 0.9868049386439646
4gru 0.9873563272245937
5cnn 0.9844012138936078
att 0.987329988998776
char_rnn 0.9889906618421405
cnnb 0.9837541668676183
conv2dm 0.9859917031900279
grucnn 0.9892953583553575
grucnn_fl 0.9884734455628387
grucnn_fl3 0.9900106258551316
grucnn_fl4 0.9897142748398743
lr 0.9864988837790026
nbsvm 0.9860737665889984
olgbm 0.980899519706738
ryanc2d 0.9856651156122117
ryangru 0.9868306224815333
wordbatch2 0.9861707682227453
ycnn 0.983265458485085
sk0 0.9908441370482972
sk1 0.9911680228358426
sk2 0.9900051360335094
sk3 0.9900499525360588
sk4 0.9903586034155518
sk5 0.9893902526807571
sk6 0.9900944701136121
sk7 0.9895366459417468
sk8 0.9906284384471021
sk9 0.9905225954504444
sk10 0.9899603886696958
chen0 0.991999076873736
chen1 0.9924723797956659
chen2 0.9919231861259981
chen3 0.9923791534572223
chen4 0.9919445448270507
chen5 0.9773253916780882
chen6 0.9918822328410242


In [55]:
train = pd.read_csv('../input/train.csv')
train = train.loc[train['id'].isin(chen_sol_ids),:].sort_values('id').reset_index(drop=True)
for o in oofs:
    train = train.merge(o, on='id', how='left')

In [56]:
'\',\''.join(onms)

"_att','2ycnn','4gru','5cnn','att','char_rnn','cnnb','conv2dm','grucnn','grucnn_fl','grucnn_fl3','grucnn_fl4','lr','nbsvm','olgbm','ryanc2d','ryangru','wordbatch2','ycnn','sk0','sk1','sk2','sk3','sk4','sk5','sk6','sk7','sk8','sk9','sk10','chen0','chen1','chen2','chen3','chen4','chen5','chen6"

In [57]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', '_att_toxic', '_att_severe_toxic',
       ...
       'chen5_obscene', 'chen5_threat', 'chen5_insult', 'chen5_identity_hate',
       'chen6_toxic', 'chen6_severe_toxic', 'chen6_obscene', 'chen6_threat',
       'chen6_insult', 'chen6_identity_hate'],
      dtype='object', length=230)

In [58]:
orig_cols = ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [59]:
scores = {}
for c in cols:
    y = train[c]
    q = train.drop(orig_cols, axis=1)
    scores[c] = []
    for n in onms:
        w = roc_auc_score(y,train[n+'_'+c])
        scores[c].append(w)
        #print(n,w)

In [60]:
len(onms)

37

In [61]:
def evaluate_nms(nms,c):
    scores = {}
    y = train[c]
    q = train.drop(orig_cols, axis=1)
    scores[c] = []
    for n in nms:
        w = roc_auc_score(y,train[n+'_'+c])
        scores[c].append(w)        
    p = []
    ws = scores[c]
    y = train[c]
    pred = 0
    i = 0
    for n in nms:
        pred += ((ws[i]-np.min(ws))/(np.max(ws)-np.min(ws))+0.01)*(minmax_scale(train[n+'_'+c],feature_range=(0.000001, 0.9999999)))
        i = i + 1
    p.append(roc_auc_score(y,pred))
    return np.mean(p)

In [62]:
best_sets = {}
for c in cols:
    bst = evaluate_nms(onms,c)
    best_set = onms
    while True:
        d = {}
        bst_j = ''
        for j in best_set:
            nms = list(set(best_set) - set([j]))
            d[j] = evaluate_nms(nms,c)
            if d[j] > bst:
                bst = d[j]
                bst_j = j
        if bst_j == '':
            break
        best_set = list(set(best_set) - set([bst_j]))
    best_sets[c] = best_set.copy()
    
ppp = []
for c in cols:
    ppp.append(evaluate_nms(best_sets[c],c))
    print(c, evaluate_nms(best_sets[c],c))
np.mean(ppp)

toxic 0.9903198606082227
severe_toxic 0.992917088394062
obscene 0.995752926720944
threat 0.9978916268544128
insult 0.9908479226321134
identity_hate 0.9946680508129225


0.9937329126704463

In [63]:
preds = []

train_files = [q for q in f if q.startswith('test')]
for q in train_files:
    nf = pd.read_csv('../blend/'+q)
    for c in cols:
        nf[c] = minmax_scale(nf[c])
    preds.append(nf)
    
sk_train = [q for q in ff if q.endswith('test_X_pred.npy')]
suf = 'sk'
i = 0
for q in sk_train:
    nf = pd.DataFrame(np.mean(np.load(q),axis=0))
    nf.columns = cols
    for c in cols:
        nf[c] = minmax_scale(nf[c])
    preds.append(nf)
    i = i + 1

suf = 'chen'
i = 0
for q in fchen:
    nf = pd.read_csv('../cheng/ensemble/9/'+q.replace('.valid','.infer')).sort_values('id').reset_index(drop=True)
    nm = suf+str(i)
    if nm not in onms:
        print(nm)
        i = i + 1
        continue
    for c in cols:
        nf[c] = minmax_scale(nf[c])
    preds.append(nf)
    i = i + 1  

In [64]:
len(preds)

37

In [65]:
sub = pd.read_csv('../input/sample_submission.csv')
for c in cols:
    sub[c] = 0
    y = train[c]
    ws = []
    for n in best_sets[c]:
        w = roc_auc_score(y,train[n+'_'+c])
        ws.append(w)
    k = 0
    for n in best_sets[c]:
        j = onms.index(n)
        sub[c] += ((ws[k]-np.min(ws))/(np.max(ws)-np.min(ws))+0.01) * preds[j][c]
        k = k + 1
    sub[c] = minmax_scale(sub[c])
sub.head(n=3)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.978505,0.477383,0.981480,0.191414,0.957730,0.522438
1,0000247867823ef7,0.005773,0.002361,0.000155,0.001484,0.000181,0.001668
2,00013b17ad220c46,0.007483,0.001864,0.000548,0.002826,0.000228,0.001077


In [66]:
sub.to_csv('weighted_blend_37models_9.csv', index=False)